In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

%matplotlib inline

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)


In [27]:
# Verificación de archivos disponibles
print("Archivos disponibles en el directorio actual:")
archivos_disponibles = [f for f in os.listdir() if f.endswith('.csv')]
for archivo in archivos_disponibles:
    print(f" - {archivo}")

Archivos disponibles en el directorio actual:
 - NBA_PBP_2017-18.csv
 - NBA_PBP_2019-20.csv
 - play_by_play.csv
 - player.csv
 - other_stats.csv
 - clutch_stats.csv
 - game_summary.csv
 - NBA_PBP_2016-17.csv
 - NBA_PBP_2018-19.csv
 - NBA_PBP_2020-21.csv
 - draft_history.csv
 - common_player_info.csv
 - NBA_PBP_2015-16.csv
 - game.csv


In [28]:
def filtrar_jugadas_clutch(df):
    df = df.copy()
    df["scoremargin"] = pd.to_numeric(df["scoremargin"], errors="coerce").astype("Int16")

    # Convertir a segundos
    # Verificar qué columna de tiempo está disponible
    time_column = "pctimestring" if "pctimestring" in df.columns else "wctimestring"

    if time_column in df.columns:
        mask_time = df[time_column].str.match(r"^\d{1,2}:\d{2}$", na=False)
        mins = pd.to_numeric(df.loc[mask_time, time_column].str.split(":").str[0], errors="coerce")
        secs = pd.to_numeric(df.loc[mask_time, time_column].str.split(":").str[1], errors="coerce")
        df.loc[mask_time, "seconds_remaining"] = mins * 60 + secs
    else:
        df["seconds_remaining"] = None
        print(f"Advertencia: No se encontró una columna de tiempo válida")

    clutch = df[
        (df["period"] >= 4) &
        (df["seconds_remaining"].notna()) &
        (df["seconds_remaining"] <= 300) &
        (df["scoremargin"].abs() <= 5)
    ].copy()

    return clutch

In [29]:
cols = [
    "game_id", "eventmsgtype", "period",
    # Incluir ambos nombres posibles para la columna de tiempo
    "pctimestring", "wctimestring",
    "scoremargin", "player1_id", "player1_name",
    "homedescription", "visitordescription"
]

dtypes = {
    "game_id": "Int64",  # Cambiado a Int64 para mayor compatibilidad
    "eventmsgtype": "Int8",  # Cambiado a Int8 para permitir valores negativos/NA
    "period": "Int8",
    "player1_id": "Int64",
    "player1_name": "object",
    "homedescription": "object",
    "visitordescription": "object"
}

In [30]:
# Intentar cargar los archivos disponibles
try:
    # Verifica si existe el archivo
    if 'NBA_PBP_2015-16.csv' in archivos_disponibles:
        nombre_archivo = 'NBA_PBP_2015-16.csv'
    elif 'NBA_PBP_2015–16.csv' in archivos_disponibles:  # Guión largo
        nombre_archivo = 'NBA_PBP_2015–16.csv'
    else:
        # Buscar cualquier archivo que contenga PBP y 2015
        archivos_pbp = [f for f in archivos_disponibles if 'PBP' in f and '2015' in f]
        nombre_archivo = archivos_pbp[0] if archivos_pbp else None

    if nombre_archivo:
        print(f"Cargando archivo: {nombre_archivo}")
        # Usar solo las columnas disponibles en el archivo
        df_1516 = pd.read_csv(nombre_archivo, dtype={k:v for k,v in dtypes.items()
                                                     if k not in ['pctimestring', 'wctimestring']})
        df_1516_clutch = filtrar_jugadas_clutch(df_1516)
        print(f"2015-16 → {len(df_1516_clutch)} jugadas clutch")
    else:
        print("No se encontró un archivo de datos compatible")
except Exception as e:
    print(f"Error al cargar el archivo: {e}")
    # Crear un pequeño dataframe de ejemplo para que podamos seguir con el código
    df_1516_clutch = pd.DataFrame(columns=["game_id", "eventmsgtype", "player1_name"])
    print("Se ha creado un dataframe vacío para continuar")


Cargando archivo: NBA_PBP_2015-16.csv
Error al cargar el archivo: 'scoremargin'
Se ha creado un dataframe vacío para continuar


KeyboardInterrupt: 